# Uncertainty estimation with deep learning for discharge prediction


This is a project for the course CEGM2003 Data Science and Artificial Intelligence for Engineers,
created by the members of the HBV Group: 
- Dwiva Anbiya Taruna (5849578)
- Konstantina Bourazani (5728347)
- Hang Long (5743702)
- Thomas Poort (4715500)

In [1]:
import pickle
import numpy as np
from pathlib import Path

import matplotlib.pyplot as plt
import torch
from neuralhydrology.evaluation import metrics
from neuralhydrology.nh_run import start_run, eval_run
from neuralhydrology.utils.nh_results_ensemble import create_results_ensemble
from neuralhydrology.nh_run_scheduler import schedule_runs
from neuralhydrology.evaluation.plots import percentile_plot, regression_plot, uncertainty_plot

## Introduction


Rainfall runoff modeling is extremely important for hydrological science as it plays an huge role in understanding and managing water resources. The water cycle, containing factors such as precipitation and evaporation, contains many interactions between the atmosphere, the land surface and the river system. Understanding each of these factors and how they impact the river systems is extremely important. Especially in current times of climate change and urbanization that continues to reshape landscapes, there is a need for accurate and reliable rainfall runoff models. In the past years it has become increasingly apparent that the climate has been changing and that river systems have been impacted greatly in the forms of droughts or floods.


<center><figure>
  <img src="Images/droughts_river_france.jpg" width=400/>
  <img src="Images/floods_river_germany.jpg" width=400/>
</figure></center>

Therefore, predicting the runoff in river systems is crucial for a multitude of applications, such as water resource management, flood risk assessment, urban planning or early warning systems. Hydrological prediction is important in effective water resource management, playing a pivotal role in addressing the challenges posed by climate variability and change. 

<center><figure>
  <img src="Images/rainfall_runoff.jpg" width=600/>
<figcaption>Rainfall-Runoff
</figure></center>

However, until now rainfall predictions have been computed deterministically without providing any information about the uncertainty that plays a significant role in these predictions. It is crucial to quantify uncertainties in rainfall-runoff predictions for multiple reasons, as it provides stakeholders a better understanding of the the possible outcoems that could occur. Therefore it is crucial for decision-making in many sectors, for example water resource management, agriculture and disaster risk reduction. 


For that reason there is a need to estimate the uncertainty by using probabilistic predictions instead of deterministic ones. The main goal of this project is to investigate different methods for uncertainty estimation in discharge prediction using probabilistic predictions.


## Objectives

The project has four objectives:

•	**Hands-on Discharge prediction:** 
    
    - Gain practical experience in discharge prediction using machine learning techniques.

•	**Uncertainty Estimation for Discharge Prediction:**

    - Use a LSTM model combined with Monte Carlo Dropout to make probabilistic predictions.

•	**Evaluation Metrics Exploration:** 

    - Probabilistic: Explore suitable metrics (PICP, PINAW) for evaluating the performance of various uncertainty estimation methods.
    - Deterministic: Using the ensemble of the MCD output, compare to the LSTM predictions.

•	**Transfer Learning:** 

    - Assess whether the obtained model can be used for Transfer Learning for different basins.




For that purpose the following workplan/flowchart was created:

<center><figure>
  <img src="Images/CEGM2003_new.png" width=600/>
<figcaption>Workplan
</figure></center>

## Data

For this project data was used from the CAMELS US dataset (Catchment Attributes and Meteorology for Large-sample Studies). This dataset contains of two main aspects; the hydrometeorological time series of each basin and the catchment attributes. 


The hydrometeorological time series contains all the meteorological data that is used as input as well as time series of streamflow observations for each basin. The catchment attributes dataset contains information about a wide range of attributes that are able to influence catchment behaviour such as topographic characteristics (slope, elevation), soil characteristics (soil depth, porosity), geological characteristics etc.

 
From this dataset that contains in total 671 catchments in the United States, one specific hydrological unit (HUC) was selected to use as input data for our model. The selected region was HUC 17 which is the North-Western area of the United States including for instance the states of Washington and Oregon.


<center><figure>
  <img src="Images/CAMELS_US_HUC.png" width=600/>
<figcaption>CAMELS US Data
</figure></center>

In [4]:
from neuralhydrology.datasetzoo.camelsus import CamelsUS, load_camels_us_attributes, load_camels_us_discharge, load_camels_us_forcings

In [32]:
# Data path
data_dir = Path("../data/CAMELS_US")

# Load the attributes
attributes_df = load_camels_us_attributes(data_dir)

# Filter for HUC 17
huc_17_df = attributes_df[attributes_df['huc'] == '17']

# Display the first few rows of the filtered DataFrame
huc_17_df.head()

,p_mean,pet_mean,p_seasonality,frac_snow,aridity,high_prec_freq,high_prec_dur,high_prec_timing,low_prec_freq,low_prec_dur,...,frac_forest,lai_max,lai_diff,gvf_max,gvf_diff,dom_land_cover_frac,dom_land_cover,root_depth_50,root_depth_99,huc
gauge_id,,,,,,,,,,,,,,,,,,,,,
10396000,1.560339,3.394634,-0.575570,0.465490,2.175574,24.00,1.307902,mam,277.90,7.236979,...,0.0045,0.758705,0.565572,0.331168,0.177819,0.999126,Grasslands,0.120052,1.500000,17
12010000,7.927943,1.967121,-0.761100,0.023636,0.248125,9.35,1.316901,djf,169.90,4.323155,...,0.9981,4.329237,2.369540,0.860577,0.133434,0.607243,Evergreen Needleleaf Forest,0.201421,2.035654,17
12013500,6.874938,2.116710,-0.799301,0.017416,0.307888,11.35,1.375758,djf,172.45,4.508497,...,0.9896,4.980796,2.745458,0.880343,0.126688,0.783664,Evergreen Needleleaf Forest,0.187307,1.929801,17
12020000,7.499240,2.739657,-0.776442,0.043553,0.365325,7.90,1.362069,djf,176.45,4.788331,...,0.9964,4.401553,2.592355,0.861865,0.139503,0.558829,Evergreen Needleleaf Forest,0.205294,2.064703,17
12025000,4.707979,2.094823,-0.780452,0.029593,0.444952,10.50,1.578947,djf,183.00,4.945946,...,0.8617,4.977506,3.118564,0.871182,0.177770,0.680921,Mixed Forests,0.225128,2.188930,17


In [33]:
# Selected Basin as example
basin_id = '14187000' # WILEY CREEK NEAR FOSTER, OR
area = 134.75*1e6

# Load the discharge data
discharge_series = load_camels_us_discharge(data_dir, basin_id, area)
print(discharge_series)

date
1988-10-01     0.179748
1988-10-02     0.156145
1988-10-03     0.148883
1988-10-04     0.152514
1988-10-05     0.176117
                ...    
2014-12-27     8.224851
2014-12-28    10.367307
2014-12-29     9.041890
2014-12-30     6.844965
2014-12-31     5.410609
Name: QObs, Length: 9588, dtype: float64


In [43]:
forcings_data = load_camels_us_forcings(data_dir, basin_id, forcings='daymet')
print(forcings_data)
forcings_data = load_camels_us_forcings(data_dir, basin_id, forcings='maurer')
print(forcings_data)
forcings_data = load_camels_us_forcings(data_dir, basin_id, forcings='nldas')
print(forcings_data)

(            Year  Mnth  Day  Hr   dayl(s)  prcp(mm/day)  srad(W/m2)  swe(mm)  \
date                                                                           
1980-01-01  1980     1    1  12  31449.60         12.21       71.02      0.0   
1980-01-02  1980     1    2  12  31449.60          5.02      111.11      0.0   
1980-01-03  1980     1    3  12  31449.60         13.07       92.87      0.0   
1980-01-04  1980     1    4  12  31480.03         10.84       55.56      0.0   
1980-01-05  1980     1    5  12  31795.19         38.25       92.26      0.0   
...          ...   ...  ...  ..       ...           ...         ...      ...   
2014-12-27  2014    12   27  12  31134.43          0.00      120.79      0.0   
2014-12-28  2014    12   28  12  31347.50         20.68       80.97      0.0   
2014-12-29  2014    12   29  12  31449.60         16.33      128.15      0.0   
2014-12-30  2014    12   30  12  31449.60          3.84      161.51      0.0   
2014-12-31  2014    12   31  12  31449.

## Base Model - LSTM

Machine learning has been widely used to produce accurate predictions for optimizing water allocation, flood control, and sustainable environmental practices. Machine learning techniques have emerged as powerful tools in the field of hydrological prediction, offering the potential to enhance accuracy and efficiency. Among these techniques, Long Short-Term Memory (LSTM) networks have gained prominence for their ability to model complex temporal dependencies in data. The application of LSTM models in hydrological prediction involves leveraging their capacity to analyze and learn patterns from historical hydrological data. By understanding the intricate relationships between various meteorological and hydrological variables, LSTMs can generate predictions that contribute to more informed decision-making in water resource management.

<center><figure>
  <img src="Images/LSTM.png" width=600/>
<figcaption>LSTM Model
</figure></center>

In [ ]:
# Brief explanation about the cudaLSTM

In [2]:
# Include some explanation for the hidden size we have chosen -> looking at training, val, test loss

## Model Head

The model head in this case means the final part of the neural network architecture responsible for generating the specific output format required for a task, which in this case can result in probabilistic modeling. In this project we look at the Monte Carlo Dropout.

### Monte Carlo Dropout (MCD)

Monte Carlo Dropout (MCD) provides a way to estimate the uncertainty of our predictions. In this case dropout is used with a certain dropout rate to randomly inactive certain neural network nodes. This results in a different network structure each time the model is evaluated during training. By repeating this process multiple times an ensemble of many sub-networks is created which all produce predictions. From this ensemble of predictions an uncertainty interval can be created and later on evaluated. 

<center><figure>
  <img src="Images/MCD.png" width=600/>
<figcaption>Monte Carlo Dropout
</figure></center>

The first step was to find a suitable dropout rate to use during the Monte Carlo Dropout process. By using different dropout rates between the range of 0.2-0.5, we tested which dropout rate would be best for our project to use.

## Training

In [ ]:
# Training part, hyperparameters chosen, any configuration in each model present here

|               | GMM                      | CMAL                     | UMAL                     | MCD                      |
|---------------|--------------------------|--------------------------|--------------------------|--------------------------|
| Training period       | 1 Jan 1980–31 Dec 1999   | 1 Jan 1980–31 Dec 1999   | 1 Jan 1980–31 Dec 1999   | 1 Jan 1980–31 Dec 1999   |
| Validation period     | 1 Jan 2000–31 Dec 2004   | 1 Jan 2000–31 Dec 2004   | 1 Jan 2000–31 Dec 2004   | 1 Jan 2000–31 Dec 2004   |
| Test period           | 1 Jan 2005–31 Dec 2009   | 1 Jan 2005–31 Dec 2009   | 1 Jan 2005–31 Dec 2009   | 1 Jan 2005–31 Dec 2009   |
| Training loss         | Negative log-likelihood  | Negative log-likelihood  | Negative log-likelihood  | MSE                      |
| CAMELS attributes     | Yes                      | Yes                      | Yes                      | Yes                      |
| Input products        | DayMet, Maurer, NLDAS    | DayMet, Maurer, NLDAS    | DayMet, Maurer, NLDAS    | DayMet, Maurer, NLDAS    |
| Regularization: noise | Yes                      | Yes                      | Yes                      | Yes                      |
| Regularization: dropout | Yes                      | Yes                      | Yes                      | Yes                      |
| Sampling space for τ  | NA                       | NA                       | NA                       | (0.01, 0.99)             |
| Gradient clipping     | Yes                      | Yes                      | No                       | Yes            | Hidden size LSTM     | 20    | 20    | 20    | 20    |
| Number of components | 10     | 3      | NA     | NA     |
| Regularization: noise| 0.2    | 0.2    | 0.2    | 0.1    |
| Regularization: dropout| 0.4  |40.5    |40.5    |2.75   |
| Batch size           | 256    | 256    | 256    | 256    |
| Learning rate        | 0.001  | 0.0005 | 0.0005 | 0.001  |           |


## Model Results

In the code above the LSTM model was trained. Now that that is achieved we can test the model performance and show the results. 

In [ ]:
# Shows model results in the same basin 

## Evaluation Model

The final step is to evaluate the model and the results obtained from the model and Monte Carlo Dropout. We do this in two ways: 

- #### Deterministic:

The Monte Carlo Dropout, shown previously, results in a thousand different prediction time-series for the discharge. It is an option to compare these deterministically to the observations and the used LSTM model without Monte Carlo Dropout. Ensembling is a way to get a deterministic result from multiple predicitons, in our case we have taken the median of all predictions and combined these into one time-series. In the end, this ensemble can be shown combined with the observations and its performance can be compared to the deterministic result we got from just using the LSTM model.

- #### Probabilistic:

One of the main objectives of our project was to see whether probabilistic predictions would be useful in making discharge predictions. As in most cases model used for discharge predictions only return deterministic results, but they do not give us an indication about the uncertainty in these predictions. Using Monte Carlo Dropout probabilistic predictions can be made. The thousand prediction time-series can be used to make a prediction interval, for which we used a 95% as the set interval. 

To evaluate these intervals we introduced two evaluation metrics, the Prediction Interval Coverage Probability (PICP) and the Prediction Interval Normalized Average Width (PINAW). The PICP is a metric that measures the percentage of target observations which are enclosed in the prediction interval. 

$$PICP = \frac{1}{n}  * \sum \limits _{i=1} ^{n} c_{i}$$


The PINAW is defined as the ratio between the average width of the prediction interval and the range of the observations.

$$PINAW = \frac{1}{nR} * \sum \limits _{i=1} ^{n} (u_{i} - l_{i})$$

Comparing these two probabilistic evaluation metric we try to find the optimal dropout rate for Monte Carlo Dropout.


In [ ]:
# Evaluation part, hyperparameters chosen, any configuration in each model present here

### Deterministic Evaluation

### Probabilistic Evaluation

## Transferability

As an extra objective, we tried to use Transfer Learning as well. Here, we used our pretrained model which we obtained previously to test it on another HUC dataset. This HUC has different characteristics than the HUC used during training. 

By using Transfer Learning we try to find out whether it is possible to use our pretrained model for other basins as well. In the code below, we test and show the results of the Transfer Learning. This examination has the aim to determine how feasible the pretrained model is for diverse basins and thereby giving insight into the capacity of the model to generalize across different environmental characteristics.


<center><figure>
  <img src="Images/Transfer_Learning.png" width=600/>
<figcaption>Transfer Learning
</figure></center>

In [ ]:
# Shows the results of probabilistic evaluation for different basin in different HUC

In [ ]:
# Model Result

In [ ]:
# Evaluation Result

## Conclusion

In [1]:
# Explain what we have seen and what we can conclude from that.

# Reflection

During the period of our project, the group walked into a few issues that caused our objectives to change. Originally, the plan was to use four uncertainty methods with which we would make probabilistic predictions for discharge estimations. These methods were the Monte Carlo Dropout, Gaussian Mixtures Model, Countable Mixture of Asymmetric Laplacians and Uncountable Mixture of Asymmetric Laplacians. 

Our approach was aimed to investigate these methodologies for uncertainty estimation. However, as we were working on the project, it became clear that our initial excitement led to us underestimating the amount of work that was required. The original workplan (see Figure below) showed our initial plan to apply these four uncertainty methods for both deterministic and probabilistic predictions. The goal was to compare these method to each other and see whether a certain method was most effective.

<center><figure>
  <img src="Images/CEGM2003.png" width=600/>
<figcaption>Previous workplan
</figure></center>

Regrettably, due to the underestimated work, it was necessary to change our objective. For that reason we decided to focus solely on the Monte Carlo Dropout method. Even though we had to change our original plan, this change ensured that we had enough time to finish the project and that the project was meaningful research into the use of probabilistic predictions.

# Project Logbook

This is the logbook we used during the project. Showing the main tasks we performed during the project and who was responsible for what. 

<center><figure>
  <img src="Images/Logbook.png" width=600/>
<figcaption>Project Logbook
</figure></center>

## References

Klotz, D., Kratzert, F., Gauch, M., Sampson, A. K., Brandstetter, J., Klambauer, G., Hochreiter, S., & Nearing, G. (2022). Uncertainty estimation with deep learning for rainfall–runoff modeling. Hydrology and Earth System Sciences, 26(6), 1673–1693. https://doi.org/10.5194/hess-26-1673-2022

Kratzert, F., Klotz, D., Brenner, C., Schulz, K., & Herrnegger, M. (2018). Rainfall–runoff modelling using Long Short-Term Memory (LSTM) networks. Hydrology and Earth System Sciences, 22(11), 6005–6022. https://doi.org/10.5194/hess-22-6005-2018

Taormina, R., & Chau, K. W. (2015). ANN-based interval forecasting of streamflow discharges using the LUBE method and MOFIPS. Engineering Applications of Artificial Intelligence, 45, 429–440. https://doi.org/10.1016/j.engappai.2015.07.019